In [1]:
import numpy as np
import pandas as pd
import random
import timeit

In [25]:
# global variables

# showfans, unique_states, N (basketsize), M (chromosomes)
# unique_index maps states to index positions in master data frame showfans
# showbaskets_d and showbaskets_r and fitness
# u_showbaskets_d and u_showbaskets_r and u_fitness and u_count

N = 4 # maxbasketsize for each 
M = 100 # number of chromosomes
ITER = 2000 # number of iterations or crossovers to run the GA for
MIN_PANEL_COUNT = 20 # threshold used to determine which states to include for fitness; set to zero for all
RANGEPROB = 0.5 # percentage of times range_crossover will be applied, other times single position crossover is used
MUTATIONPROB = 0.1 # 
MUTATIONAMOUNT = 2 # how many genes to mutate when mutation is run

In [26]:
state_margins = pd.read_csv('ElectionRsults-2016-D-PCNT.csv')

In [27]:
def d_margin(state_margins, state):
    # the margin is a number such as 22.1, interpreted as D% is greater than R% by 22.1%
    return(float(state_margins[state_margins.State==state].iloc[0]['Dem 16 Margin'].strip('%')))
    # call as d_margin(state_margins, 'NEW YORK')

In [28]:
showfansALL = pd.read_csv('subset-data.csv', encoding = "ISO-8859-1")

In [29]:
def select_subset_shows():
    # we can modify this as needed to select more/less shows
    x=showfansALL.groupby('State').sum()
    cols = x.columns
    new = pd.DataFrame()
    for i in cols:
        if x.loc['FLORIDA'][i] > 20:
            new[i]=x[i]
    return new.columns

In [30]:
# CREATE SUBSET DATAFRAME showfans
newshows = select_subset_shows()
showfans = pd.DataFrame()
cols = showfansALL.columns
showfans[cols[0]]=showfansALL[cols[0]]
for i in newshows:
    showfans[i] = showfansALL[i]


In [31]:
length = len(showfans.index)
cols = len(showfans.columns)
print(length, cols)

1999 96


In [32]:
# CREATE UNIQUE STATES LIST AND INDEX LIST TO USE LATER 

unique_states = showfans['State'].unique()

unique_index = dict()

for state in unique_states:
    unique_index[state] = showfans.loc[showfans['State'] == state].index
    

In [33]:
# create state_array of length 2000 storing state values of all panelists
# create 96 arrays of length 2000 storing boolean show-likes
# this significantly speeds up fitness calculations

# THIS BLOCK CAN TAKE ONE MINUTE TO RUN 
state_array = np.empty(length,dtype = "S22")
for i in range(length):
    state_array[i]=showfans.iloc[i][0]
    
watch_array = np.empty([length,cols-1], dtype = bool)
for i in range(length):
    for j in range(cols-1):
        watch_array[i,j] = showfans.iloc[i,j+1]

In [34]:
# genetic algorithm chromosome and fitness initialization

showbaskets_D = np.empty([M,N],dtype=int)
showbaskets_R = np.empty([M,N],dtype=int)
#fitness array below should store the fitness value of each chromosome
fitness=np.empty(M,dtype=float)

# below will have unique elements, will only compute and use when needed
u_showbaskets_D = np.empty([M,N],dtype=int)
u_showbaskets_R = np.empty([M,N],dtype=int)
u_fitness=np.empty(M,dtype=float)
for i in range(M):
    u_fitness[i]=0.0
u_count = 0

In [35]:
def calculate_uniques():
    # use showbaskets chromosomes to find unique values
    global u_count
    u_count = 0
    
    for i in range(N):
        u_showbaskets_D[u_count,i] = showbaskets_D[0,i]
        u_showbaskets_R[u_count,i] = showbaskets_R[0,i]
    u_fitness[u_count]=fitness[0]
    u_count += 1
    
    for j in range(M-1):
        if isnt_in(j+1):
            for i in range(N):
                u_showbaskets_D[u_count,i] = showbaskets_D[j+1,i]
                u_showbaskets_R[u_count,i] = showbaskets_R[j+1,i]
            u_fitness[u_count] = fitness[j+1]
            u_count += 1

def isnt_in(pos):
    d1 = np.unique(showbaskets_D[pos])
    r1 = np.unique(showbaskets_R[pos])
    for i in range(pos):
        d2 = np.unique(u_showbaskets_D[i])
        r2 = np.unique(u_showbaskets_R[i])
        if np.array_equal(d1,d2) and np.array_equal(r1,r2):
            return 0
    return 1
        

In [13]:
def calc_fitness(chrome):
    #calculate fitness of chrome-th chromosome based on how good the basket assignment is    
    #for each state, create random subset of indexes first
    #for those indexes, apply basket rules and do counts D and R and percentages
    #compare percentages with actuals and derive error value
    
    fit = 0.0 # need to add fitness for each state and then calculate average fitness
   
    THRESH = 1.1 # used to compare D and R show count of a person to decide how to label each guy
    
    ignore = 0 # used to decide which states to ignore baseed on MIN_PANEL_COUNT
    
    for state in unique_states:
        x = unique_index[state]
        if (len(x) < MIN_PANEL_COUNT):
            ignore += 1
            continue
        samples = len(x)
        TOTDCOUNT = 0.0
        TOTRCOUNT = 0.0
        
        for pos in x:
            DCOUNT = 1.0
            RCOUNT = 1.0
            
            for show in range(N):
                if watch_array[pos,showbaskets_D[chrome,show]-1]:
                    DCOUNT += 1
                if watch_array[pos,showbaskets_R[chrome,show]-1]:
                    RCOUNT += 1
    
            if (DCOUNT/RCOUNT) > THRESH:
                TOTDCOUNT += 1
            elif (RCOUNT/DCOUNT) > THRESH:
                TOTRCOUNT += 1
                
        # completed iterating over all samples for this state
        predict_DMARGIN = (TOTDCOUNT/samples)-(TOTRCOUNT/samples)
        actual_DMARGIN = d_margin(state_margins, state)/100
        #print(state," has ",len(x)," panelists. Fit ","pred ",predict_DMARGIN," actual ",actual_DMARGIN)
        #print("Percentage of people we matched is ", (TOTDCOUNT+TOTRCOUNT)/samples)
        #fit += ((TOTDCOUNT+TOTRCOUNT)/samples)*(1-abs(predict_DMARGIN - actual_DMARGIN))
        fit += (1-abs(predict_DMARGIN - actual_DMARGIN))
    fit = fit/(len(unique_states)-ignore) # calculates average fitness across all states
    return fit
    


In [36]:
#INITIALIZATION OF POPULATION
def initialize():
    for chrome in range(M):
        set = random.sample(range(1,cols),2*N)
        for i in range(N):
            showbaskets_D[chrome,i] = set[i]
            showbaskets_R[chrome,i] = set[i+N]
        fitness[chrome] = calc_fitness(chrome)
        print("Fitness of chrome ",chrome," is",fitness[chrome])

In [37]:
def random_fitness_weighted_index():
    total=sum(fitness)
    x = random.uniform(0,total)
    done = current_sum = i = 0
    
    while (done != 1):
        current_sum = current_sum + fitness[i]
        if (current_sum >= x):
            done = 1
        else:
            i=i+1
    return i

In [38]:
def is_present(d,r):
    #assumes d and r are arrays representing 
    #democratic and rep baskets of size N
    for i in range(M):
        if np.array_equal(d,showbaskets_D[i]) and np.array_equal(r,showbaskets_R[i]):
            return 1
    return 0
    

In [39]:
def single_position_crossover():
    
    child1 = np.empty([N],dtype=int)
    child2 = np.empty([N],dtype=int)
    
    #pick two rows for crossover proportional to their fitness
    row1 = random_fitness_weighted_index()
    row2 = random_fitness_weighted_index()
    while (row2 == row1):
        row2 = random_fitness_weighted_index()
    
    k1 = random.randint(0,N-1) # crossover point 
    
    sorted_indices = np.argsort(fitness)

    dswap=0
    
    if (random.uniform(0,1) < 0.5):
        dswap = 1
        #d_chrome
        if (showbaskets_D[row2, k1] in showbaskets_D[row1]) or (showbaskets_D[row2, k1] in showbaskets_R[row1]):
            return
        if (showbaskets_D[row1, k1] in showbaskets_D[row2]) or (showbaskets_D[row1, k1] in showbaskets_R[row2]):
            return
        for i in range(N):
            child1[i] = showbaskets_D[row1,i]
            child2[i] = showbaskets_D[row2,i]
        temp = child1[k1]
        child1[k1]=child2[k1]
        child2[k1]=temp
        # overwrite the worst two chromosomes with the new children
        if (is_present(child1,showbaskets_R[row1])) and (is_present(child2,showbaskets_R[row2])):
            return

        for i in range(N):
            showbaskets_D[sorted_indices[0],i]=child1[i]
            showbaskets_R[sorted_indices[0],i]=showbaskets_R[row1,i]
            showbaskets_D[sorted_indices[1],i]=child2[i]
            showbaskets_R[sorted_indices[1],i]=showbaskets_R[row2,i]
            
    else:
        if (showbaskets_R[row2, k1] in showbaskets_R[row1]) or (showbaskets_R[row2, k1] in showbaskets_D[row1]):
            return
        if (showbaskets_R[row1, k1] in showbaskets_R[row2]) or (showbaskets_R[row1, k1] in showbaskets_D[row2]):
            return
        for i in range(N):
            child1[i] = showbaskets_R[row1,i]
            child2[i] = showbaskets_R[row2,i]
        temp = child1[k1]
        child1[k1]=child2[k1]
        child2[k1]=temp
        if (is_present(showbaskets_D[row1], child1) and is_present(showbaskets_D[row2], child2)):
            return
        # overwrite the worst two chromosomes with the new children

        for i in range(N):
            showbaskets_R[sorted_indices[0],i]=child1[i]
            showbaskets_D[sorted_indices[0],i]=showbaskets_D[row1,i]
            showbaskets_R[sorted_indices[1],i]=child2[i]
            showbaskets_D[sorted_indices[1],i]=showbaskets_D[row2,i]

    fitness[sorted_indices[0]] = calc_fitness(sorted_indices[0])
    
    fitness[sorted_indices[1]] = calc_fitness(sorted_indices[1])


In [40]:
def range_crossover():
    child1 = np.empty([N],dtype=int)
    child2 = np.empty([N],dtype=int)
    
    #pick two rows for crossover proportional to their fitness
    row1 = random_fitness_weighted_index()
    row2 = random_fitness_weighted_index()
    while (row2 == row1):
        row2 = random_fitness_weighted_index()
    
    sorted_indices = np.argsort(fitness)

    dswap=0
    
    k1 = random.randint(0,N-1) # crossover point 
    
    if (random.uniform(0,1) < 0.5):
        dswap = 1
        #d_chrome
        for i in range(k1):
            child1[i]=showbaskets_D[row1, i]
            child2[i]=showbaskets_D[row2, i]
        
        for i in range(k1,N):
            child1[i]=showbaskets_D[row2, i]
            child2[i]=showbaskets_D[row1, i]
            
        # overwrite the worst two chromosomes with the new children
        if (is_present(child1,showbaskets_R[row1])) and (is_present(child2,showbaskets_R[row2])):
            return
        
        for i in range(N):
            showbaskets_D[sorted_indices[0],i]=child1[i]
            showbaskets_R[sorted_indices[0],i]=showbaskets_R[row1,i]
            showbaskets_D[sorted_indices[1],i]=child2[i]
            showbaskets_R[sorted_indices[1],i]=showbaskets_R[row2,i]
    else:
        #r_chrome
        for i in range(k1):
            child1[i]=showbaskets_R[row1, i]
            child2[i]=showbaskets_R[row2, i]
        
        for i in range(k1,N):
            child1[i]=showbaskets_R[row2, i]
            child2[i]=showbaskets_R[row1, i]
            
        if (is_present(showbaskets_D[row1], child1) and is_present(showbaskets_D[row2], child2)):
            return
        
        # overwrite the worst two chromosomes with the new children
        for i in range(N):
            showbaskets_R[sorted_indices[0],i]=child1[i]
            showbaskets_D[sorted_indices[0],i]=showbaskets_D[row1,i]
            showbaskets_R[sorted_indices[1],i]=child2[i]
            showbaskets_D[sorted_indices[1],i]=showbaskets_D[row2,i]

    fitness[sorted_indices[0]] = calc_fitness(sorted_indices[0])
    
    fitness[sorted_indices[1]] = calc_fitness(sorted_indices[1])


In [41]:
def mutate():
    sorted_indices = np.argsort(fitness)
    # pick any chromosome and mutate some genes each in D and R
    chrome = random.randint(1,M)
    
    # copy over the chromosome first into a random spot which is not the "best 10"
    # to avoid overwriting the best 10
    overwrite_spot = random.randint(0,M-11)
    for i in range(N):
        showbaskets_D[sorted_indices[overwrite_spot],i]=showbaskets_D[sorted_indices[M-1-(chrome-1)],i]
        showbaskets_R[sorted_indices[overwrite_spot],i]=showbaskets_R[sorted_indices[M-1-(chrome-1)],i]
    
    for i in range(MUTATIONAMOUNT):
        set = random.sample(range(1,cols),2)
        pos = random.randint(1,N)
        showbaskets_D[sorted_indices[overwrite_spot],pos-1] = set[0]
        pos = random.randint(1,N)                                                    
        showbaskets_R[sorted_indices[overwrite_spot],pos-1] = set[1]
    
    fitness[sorted_indices[overwrite_spot]] = calc_fitness(sorted_indices[overwrite_spot])
                                                         
                                                         

In [42]:
def print_interim_output(i):
    print("======= INTERIM OUTPUT of ITER ", i+1," ======")
    print("Average fitness of population now is ", sum(fitness)/M)
    sorted_indices = np.argsort(fitness)
    print("Best fitness currently is ", fitness[sorted_indices[M-1]])
    print("Best D-BASKET: ",end="")
    for j in range(N):
        shownum = showbaskets_D[sorted_indices[M-1],j]
        print(newshows[shownum-1]," ",end="")
    print("\nBest R-BASKET: ",end="")
    for j in range(N):
        shownum = showbaskets_R[sorted_indices[M-1],j]
        print(newshows[shownum-1]," ",end="")
    print("\n")
    print("Number of likely unique chromosomes is: ",len(np.unique(fitness)))

In [43]:
def print_final_output():
    print("========= FINAL OUTPUT ===============")
    print_interim_output(ITER)
    calculate_uniques()
    print("------ UNIQUE LIST OF FINAL ",u_count," CHROMOSOMES BELOW -------")
    sorted_indices = np.argsort(u_fitness)
    for i in range(u_count):
        print(".....FITNESS OF CHROMOSOME BELOW IS ", u_fitness[sorted_indices[M-i-1]])
        print("D-BASKET: ",end="")
        for j in range(N):
            shownum = u_showbaskets_D[sorted_indices[M-i-1],j]
            print(newshows[shownum-1]," ",end="")
        print("\nR-BASKET: ",end="")
        for j in range(N):
            shownum = u_showbaskets_R[sorted_indices[M-i-1],j]
            print(newshows[shownum-1]," ",end="")
        print("\n")
    print("==================================================")
    

In [44]:
def boost_diversity():
    sorted_indices = np.argsort(fitness)
    previous_best_fitness = fitness[sorted_indices[M-1]]
    print("________________________________________________")
    print("Boosting diversity by regenerating worst 20%")
    
    for j in range(20):
        #chrome[sorted_indices[j]] is random again now
        set = random.sample(range(1,cols),2*N)
        for i in range(N):
            showbaskets_D[sorted_indices[j],i] = set[i]
            showbaskets_R[sorted_indices[j],i] = set[i+N]
        fitness[sorted_indices[j]] = calc_fitness(sorted_indices[j])
        
    sorted_indices = np.argsort(fitness)
    #sometimes diversity booster may create a better chromosome, if so
    #we want to know it, which is why we are doing this compute below
    current_best_fitness = fitness[sorted_indices[M-1]]
    if (current_best_fitness > previous_best_fitness):
        print("Diversity boosting generated a DIAMOND of fitness ", current_best_fitness)
            
    print("________________________________________________")

In [45]:

# EXECUTION STARTS HERE

start_time = timeit.default_timer()

initialize()

end_time = timeit.default_timer()    

print("Time taken to initialize ",M," chromosomes is ", end_time - start_time)


Fitness of chrome  0  is 0.8022321647541121
Fitness of chrome  1  is 0.8274343002226352
Fitness of chrome  2  is 0.7275886119671582
Fitness of chrome  3  is 0.7591383069616341
Fitness of chrome  4  is 0.8501585259556229
Fitness of chrome  5  is 0.768022992400206
Fitness of chrome  6  is 0.8390386749954979
Fitness of chrome  7  is 0.8594965314174428
Fitness of chrome  8  is 0.7765405798078842
Fitness of chrome  9  is 0.8206128506763325
Fitness of chrome  10  is 0.8099986960076463
Fitness of chrome  11  is 0.7098454916461622
Fitness of chrome  12  is 0.7068175170068011
Fitness of chrome  13  is 0.7202410226644625
Fitness of chrome  14  is 0.860427256123089
Fitness of chrome  15  is 0.7993472838402343
Fitness of chrome  16  is 0.8195989158411956
Fitness of chrome  17  is 0.8173748989469256
Fitness of chrome  18  is 0.7756657492912152
Fitness of chrome  19  is 0.8435240568031935
Fitness of chrome  20  is 0.6834949453936547
Fitness of chrome  21  is 0.8057366399059885
Fitness of chrome  22 

In [46]:
# the "REAL" Genetic Algorithm

for i in range(ITER):
    
    if (i%100 == 0):
        print_interim_output(i)
        
    if (i>0) and (i%300 == 0):
        boost_diversity()
    
    if (random.uniform(0,1) < 0.7):
        range_crossover()
    else:
        single_position_crossover()
        
    if (random.uniform(0,1) < MUTATIONPROB):
        mutate()
                                
print_final_output()                        

======= INTERIM OUTPUT of ITER  1  ======
Average fitness of population now is  0.7591133528391262
Best fitness currently is  0.860427256123089
Best D-BASKET: ClevelandShow  Cops  Castle  Middle  
Best R-BASKET: CNNErinBurnett  Colbert  CNNJakeTapper  AmericanPickers  

Number of likely unique chromosomes is:  100
======= INTERIM OUTPUT of ITER  101  ======
Average fitness of population now is  0.8449407741159334
Best fitness currently is  0.8928969032571545
Best D-BASKET: ClevelandShow  ChicagoFire  Castle  SharkTank  
Best R-BASKET: RealHousewives  DaysLives  AmericanPickers  Corden  

Number of likely unique chromosomes is:  99
======= INTERIM OUTPUT of ITER  201  ======
Average fitness of population now is  0.8650509538978457
Best fitness currently is  0.9020507190769412
Best D-BASKET: Cops  TonightShow  Elementary  ImpracticalJokers  
Best R-BASKET: CollegeFootball  FoxFriends  AmericanPickers  Corden  

Number of likely unique chromosomes is:  97
======= INTERIM OUTPUT of ITER  3

In [ ]:
print_final_output()